# Spark Join Examples

The following code provides examples of different join scenarios that can be run in Spark with Scala. The DataFrames created and joined in these examples reference the [2017 NBA All-Star starting 5 players](https://www.cbssports.com/nba/news/nba-all-star-game-2017-rosters-lineup-starters-reserves/) from both the Eastern Conference and the Western Conference. These are very small DataFrames that can be easily created and joined for example purposes. But the power of Spark lies in its ability to join much larger DataFrames at scale.

Let's start by creating a DataFrame of the All-Star game's starting players, their team ids, and the position ids for the positions that they play:

In [1]:
val sqlC = new org.apache.spark.sql.SQLContext(sc)
import sqlC.implicits._


//Create the players DataFrame
val players = Seq(
    (0, "Kyrie Irving", 2, Seq(1)),
    (1, "DeMar DeRozan", 4, Seq(2)),
    (2, "LeBron James", 2, Seq(3, 4, 5)),
    (3, "G. Antetokounmpo", 0, Seq(3, 4, 5)),
    (4, "Jimmy Butler", 1, Seq(3, 4, 5)),
    (5, "Stephen Curry", 6, Seq(1)),
    (6, "James Harden", 10, Seq(2)),
    (7, "Kevin Durant", 6, Seq(3)),
    (8, "Kawhi Leonard", 3, Seq(4)),
    (9, "Anthony Davis", 8, Seq(5))).
  toDF("id", "name", "team_id", "position")

players.show()

+---+----------------+-------+---------+
| id|            name|team_id| position|
+---+----------------+-------+---------+
|  0|    Kyrie Irving|      2|      [1]|
|  1|   DeMar DeRozan|      4|      [2]|
|  2|    LeBron James|      2|[3, 4, 5]|
|  3|G. Antetokounmpo|      0|[3, 4, 5]|
|  4|    Jimmy Butler|      1|[3, 4, 5]|
|  5|   Stephen Curry|      6|      [1]|
|  6|    James Harden|     10|      [2]|
|  7|    Kevin Durant|      6|      [3]|
|  8|   Kawhi Leonard|      3|      [4]|
|  9|   Anthony Davis|      8|      [5]|
+---+----------------+-------+---------+



Now we will create a new DataFrame containing team names and the conference that they play in:

In [2]:
//Create the teams DataFrame
val teams = Seq(
    (0, "Bucks", "Eastern"),
    (1, "Bulls", "Eastern"),
    (2, "Cavaliers", "Eastern"),
    (3, "Spurs", "Western"),
    (4, "Raptors", "Eastern"),
    (5, "Magic", "Eastern"),
    (6, "Warriors", "Western"),
    (7, "Celtics", "Eastern"),
    (8, "Pelicans", "Western"),
    (9, "Lakers", "Western"),
    (10, "Rockets", "Western")).
  toDF("id", "team", "conference")

teams.show()

+---+---------+----------+
| id|     team|conference|
+---+---------+----------+
|  0|    Bucks|   Eastern|
|  1|    Bulls|   Eastern|
|  2|Cavaliers|   Eastern|
|  3|    Spurs|   Western|
|  4|  Raptors|   Eastern|
|  5|    Magic|   Eastern|
|  6| Warriors|   Western|
|  7|  Celtics|   Eastern|
|  8| Pelicans|   Western|
|  9|   Lakers|   Western|
| 10|  Rockets|   Western|
+---+---------+----------+



Finally, we'll create another small DataFrame that lists all of the different basketball positions:

In [3]:
//Create the positions DataFrame
val positions = Seq(
    (1, "Point Gaurd"),
    (2, "Shooting Gaurd"),
    (3, "Small Forward"),
    (4, "Power Forward"),
    (5, "Center")).
  toDF("id", "position")

positions.show()

+---+--------------+
| id|      position|
+---+--------------+
|  1|   Point Gaurd|
|  2|Shooting Gaurd|
|  3| Small Forward|
|  4| Power Forward|
|  5|        Center|
+---+--------------+



### Inner-Join

The first join we'll create is an inner join that links the player names to their respective teams and conferences. Notice the `.drop` function used below. Since the `players` and the `teams` DataFrames both have a column names *id*, we will drop the `teams` copy of the column. If we did not do this, we would have two columns named *id*.

In [4]:
//Join the data from the teams DataFrame to the players DataFrame

val joinExpr = players.col("team_id") === teams.col("id")

players.join(teams, joinExpr, "inner").
    drop(teams.col("id")).
    show()

+---+----------------+-------+---------+---------+----------+
| id|            name|team_id| position|     team|conference|
+---+----------------+-------+---------+---------+----------+
|  0|    Kyrie Irving|      2|      [1]|Cavaliers|   Eastern|
|  1|   DeMar DeRozan|      4|      [2]|  Raptors|   Eastern|
|  2|    LeBron James|      2|[3, 4, 5]|Cavaliers|   Eastern|
|  3|G. Antetokounmpo|      0|[3, 4, 5]|    Bucks|   Eastern|
|  4|    Jimmy Butler|      1|[3, 4, 5]|    Bulls|   Eastern|
|  5|   Stephen Curry|      6|      [1]| Warriors|   Western|
|  6|    James Harden|     10|      [2]|  Rockets|   Western|
|  7|    Kevin Durant|      6|      [3]| Warriors|   Western|
|  8|   Kawhi Leonard|      3|      [4]|    Spurs|   Western|
|  9|   Anthony Davis|      8|      [5]| Pelicans|   Western|
+---+----------------+-------+---------+---------+----------+



### Outer-Join

Now we'll create an outer-join that connects the players to their respective teams and retains the teams that do not have any players in the All-Star game starting line-ups. Notice that `null` values are used to indicate that no player data exists for the three teams without players.

In [7]:
//Outer-Join the data from the players DataFrame to the teams DataFrame

//This shows all the team names in the DataFrame
//even without players that are started the All-Star game

teams.join(players, joinExpr, "left_outer").
    drop(teams.col("id")).
    show()

+---------+----------+----+----------------+-------+---------+
|     team|conference|  id|            name|team_id| position|
+---------+----------+----+----------------+-------+---------+
|    Bucks|   Eastern|   3|G. Antetokounmpo|      0|[3, 4, 5]|
|    Bulls|   Eastern|   4|    Jimmy Butler|      1|[3, 4, 5]|
|Cavaliers|   Eastern|   2|    LeBron James|      2|[3, 4, 5]|
|Cavaliers|   Eastern|   0|    Kyrie Irving|      2|      [1]|
|    Spurs|   Western|   8|   Kawhi Leonard|      3|      [4]|
|  Raptors|   Eastern|   1|   DeMar DeRozan|      4|      [2]|
|    Magic|   Eastern|null|            null|   null|     null|
| Warriors|   Western|   7|    Kevin Durant|      6|      [3]|
| Warriors|   Western|   5|   Stephen Curry|      6|      [1]|
|  Celtics|   Eastern|null|            null|   null|     null|
| Pelicans|   Western|   9|   Anthony Davis|      8|      [5]|
|   Lakers|   Western|null|            null|   null|     null|
|  Rockets|   Western|   6|    James Harden|     10|   

### Anti-Join

Semi-joins only retain the information from the primary DataFrame after the join. You can think of it as a type of filter. An *anti-join* is a type of semi-join that retains the values that could **not** be joined to the secondary table.

In the case below, we only want to look at which teams in our `teams` DataFrame do not have any players in starting in the NBA All-Star game.

In [5]:
//Semi-Join the data from the teams DataFrame to the players DataFrame

//This shows only the teams that did not have any starters in the All-Star game

teams.join(players, joinExpr, "left_anti").show()

+---+-------+----------+
| id|   team|conference|
+---+-------+----------+
|  5|  Magic|   Eastern|
|  7|Celtics|   Eastern|
|  9| Lakers|   Western|
+---+-------+----------+



### Complex Joins

The code below shows how to use each player's *position* array to connect them to all of their position descriptions in the `positions` DataFrame. The Eastern Conference had 3 starters that play the Small Forward position. Since a team needs to have a starter in each position, those 3 players are eligible to play Small Forward, Power Forward, and Center.

In [1]:
//A complex-join that lists all possible positions for each of the All-Star game starters

import org.apache.spark.sql.functions.expr

players.
    withColumnRenamed("position", "position_id").
    drop(players.col("id")).
    join(positions, expr("array_contains(position_id, id)")).
    show()

+----------------+-------+-----------+---+--------------+
|            name|team_id|position_id| id|      position|
+----------------+-------+-----------+---+--------------+
|    Kyrie Irving|      2|        [1]|  1|   Point Gaurd|
|   DeMar DeRozan|      4|        [2]|  2|Shooting Gaurd|
|    LeBron James|      2|  [3, 4, 5]|  3| Small Forward|
|    LeBron James|      2|  [3, 4, 5]|  4| Power Forward|
|    LeBron James|      2|  [3, 4, 5]|  5|        Center|
|G. Antetokounmpo|      0|  [3, 4, 5]|  3| Small Forward|
|G. Antetokounmpo|      0|  [3, 4, 5]|  4| Power Forward|
|G. Antetokounmpo|      0|  [3, 4, 5]|  5|        Center|
|    Jimmy Butler|      1|  [3, 4, 5]|  3| Small Forward|
|    Jimmy Butler|      1|  [3, 4, 5]|  4| Power Forward|
|    Jimmy Butler|      1|  [3, 4, 5]|  5|        Center|
|   Stephen Curry|      6|        [1]|  1|   Point Gaurd|
|    James Harden|     10|        [2]|  2|Shooting Gaurd|
|    Kevin Durant|      6|        [3]|  3| Small Forward|
|   Kawhi Leon